In [1]:
# Import all the necessary libraries/modules
import pandas as pd
import numpy as np
import requests   # Optional
from bs4 import BeautifulSoup    # Optionalt
from datetime import datetime
import sqlite3

In [2]:
# # Function to log progress messages with timestamps
def log_progress(message):
    timestamp_format = "%Y-%m-%d-%H:%M:%S"    # Year-month-day-Hour:Minute:Second
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    
    with open("code_log.txt", "a") as f:      # Append mode to keep a continuous log
        f.write(f"{timestamp}: {message} \n")

## TASK 1 - EXTRACTION

In [3]:
# Function to extract data from the URL
def extract(url):
    tables = pd.read_html(url)    # Read HTML tables into a list of DataFrames
    df = tables[1]                # Select the second table (index 1)
    
    return df

**The extraction task is pretty small by the way...**

## TASK 2 - TRANSFORMATION

In [4]:
# # Function to transform the extracted data
def transform(df):

    df = df.iloc[:, 1:]    # Keep only the necessary columns (excluding the first)
    
    # Rename columns for clarity
    df.columns = ["Name", "MC_USD_Billion"]    # MC stands for Market Cap 
    
    # Convert market cap to GBP, EUR, and INR
    df["MC_GBP_Billion"] = round(df["MC_USD_Billion"] * 0.8, 2)
    df["MC_EUR_Billion"] = round(df["MC_USD_Billion"] * 0.93, 2)
    df["MC_INR_Billion"] = round(df["MC_USD_Billion"] * 82.95, 2)
    
    # Our final columns will look something like this - Name, MC_USD_Billion, MC_GBP_Billion, MC_EUR_Billion, MC_INR_Billion\
    
    return df

**Not that hard...**

## TASK 3 - LOADING

In [5]:
# Function to load data into a CSV file
def load_to_csv(df, csv_file_name):
    df.to_csv(csv_file_name, index=False)    # Save DataFrame to CSV without row indices
    
# Function to load data into a database
def load_to_db(df, db_name, tb_name):
    conn = sqlite3.connect(db_name)          # Connect to the SQLite database
    df.to_sql(tb_name, conn, if_exists="replace", index=False)    # Replace table if it exists

**Our ETL Job is actually finished, but we gotta make a function which will run some queries and can get some information out for us from the database**

In [6]:
# Function to run a query on the database and print the results
def run_query(query_statement, db_name):
    conn = sqlite3.connect(db_name)          # Connect to the SQLite database
    query_output = pd.read_sql(query_statement, conn)    # Execute the query and read the results into a DataFrame
    print(query_statement)
    print(query_output)
    conn.close()       # Close the database connection

**Let's call out all the functions and combine all of 'em with `log_progress()` in order to complete our ETL Job**

In [7]:
# Let's call out all the functions and combine all of them with log_progress() in order to complete our ETL Job
# Start the ETL job
log_progress("ETL Job has started")

# Extraction Phase
log_progress("Phase 1 Extraction has started")
url = "https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks"
df = extract(url)
log_progress("Phase 1 Extraction has ended")

# Transformation Phase
log_progress("Phase 2 Transformation has started")
df = transform(df)
log_progress("Phase 2 Transformation has ended")

# Loading Phase
log_progress("Phase 3 Loading has started")

# Load data to csv
csv_file_name = "largest_banks_data.csv"
load_to_csv(df, csv_file_name)

# Load data to database 
db_name = "banks.db"
tb_name = "largest_banks"
load_to_db(df, db_name, tb_name)

log_progress("Phase 3 Loading has Ended")

log_progress("Starting the running query")

# Start Running queries
db_name = "banks.db"
tb_name = "largest_banks"

# Run query to select all data
query_statement = f"SELECT * FROM {tb_name}"
run_query(query_statement, db_name)
print("\n")
print("\n")

# Run query to get the average market cap in GBP
query_statement = f"SELECT AVG(MC_GBP_Billion) FROM {tb_name}"
run_query(query_statement, db_name)
print("\n")
print("\n")

# Run query to get the names of the first 5 banks
query_statement = f"SELECT Name from {tb_name} LIMIT 5"
run_query(query_statement, db_name)

log_progress("Ending the running query")

# End the ETL job
log_progress("ETL Job has ended")
# -------------------------------------------------------
# Our ETL Job is done above this line.

SELECT * FROM largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    

### Note to Viewers

Check out the `largest_banks_data.csv` and `banks.db` to see the output of our ETL Job. Here you'll find the top 10 banks with their market cap in various currencies like USD, GBP, EUR, and INR. Also, review the `code_log.txt` to see the timestamps for each phase of the ETL Job.

# THANK YOU :)